# EDA
Download data from binance

In [ ]:
from datetime import datetime, timezone
from pathlib import Path

FAPI_BASE = "https://fapi.binance.com"

RAW_DIR = Path("data/raw")
PROCESSED_DIR = Path("data/processed")
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
from importlib import reload
import utils.binance_download as binance_download
reload(binance_download)
from utils.binance_download import BinancePublic, save_df, parse_date

In [ ]:

SYMBOL = "BTCUSDT"
INTERVAL = "1h"
START = "2023-01-01"
END = None

RUN_KLINES   = True
RUN_PREMIUM  = True
RUN_FUNDING  = True
RUN_TAKER    = True
RUN_OI       = True

dl = BinancePublic()

In [ ]:
bp = BinancePublic()

start_dt = parse_date(START)
end_dt   = parse_date(END)

# There is no "1m" for the taker/oi period on Binance → replace with "1h"
_period = INTERVAL if INTERVAL in {"5m","15m","30m","1h","2h","4h","6h","12h","1d"} else "1h"

if RUN_KLINES:
    k1 = bp.fetch_klines(SYMBOL, INTERVAL, start=start_dt, end=end_dt)
    save_df(k1, RAW_DIR / f"{SYMBOL}_fapi_klines_{INTERVAL}.csv")
    display(k1.tail())

if RUN_PREMIUM:
    p1 = bp.fetch_premium_index_klines(SYMBOL, INTERVAL, start=start_dt, end=end_dt)
    save_df(p1, RAW_DIR / f"{SYMBOL}_premiumIndexKlines_{INTERVAL}.csv")
    display(p1.tail())

if RUN_FUNDING:
    f1 = bp.fetch_funding_rates(SYMBOL, start=start_dt, end=end_dt)
    save_df(f1, RAW_DIR / f"{SYMBOL}_fundingRate.csv")
    display(f1.tail())

if RUN_TAKER:
    # taker buy/sell volume (история ~30 дней)
    t1 = bp.fetch_taker_buy_sell_volume(SYMBOL, period=_period, limit=500)
    save_df(t1, RAW_DIR / f"{SYMBOL}_takerBuySellVol_{_period}.csv")
    display(t1.tail())

if RUN_OI:
    oin = bp.fetch_open_interest_now(SYMBOL)
    save_df(oin, RAW_DIR / f"{SYMBOL}_openInterest_now.csv")
    oih = bp.fetch_open_interest_hist(SYMBOL, period=_period, limit=500)
    save_df(oih, RAW_DIR / f"{SYMBOL}_openInterestHist_{_period}.csv")
    display(oih.tail())
